In [14]:
import pandas as pd
import numpy as np

df = pd.read_csv('columbia_full.csv')
df.columns

Index(['FechaPlanInicioTransporte', 'RegionalDistribucion', 'IDCentro',
       'CodigoCliente', 'DocumentoTransporte', 'Ordered_vol', 'ordered_boxes',
       'Plan_Vol', 'plan_boxes'],
      dtype='object')

In [9]:
regions = pd.unique(df['RegionalDistribucion'])
n_centres = len(pd.unique(df['IDCentro']))
print(f"Regions: {regions}, Number of centres: {n_centres}")

Regions: ['Sur' 'Centro' 'Norte' 'Andes'], Number of centres: 64


In [18]:
df.groupby('CodigoCliente').size().sort_values(ascending=False) # Number of orders

CodigoCliente
13937684    231
10359686    112
11430868     97
10151328     81
10316879     69
           ... 
12596356      1
12596294      1
12596293      1
12596275      1
14323128      1
Length: 217343, dtype: int64

In [17]:
df.groupby('CodigoCliente')['ordered_boxes'].mean().sort_values(ascending=False).head(10)

CodigoCliente
14048190    5100.000000
12719444    3840.000000
12291643    3756.500000
11618884    3718.380952
12549492    3240.000000
12404538    3213.772727
12291291    3202.000000
12282136    3136.800000
13778771    3072.000000
13876309    3000.000000
Name: ordered_boxes, dtype: float64